# COBRApy

COBRApy is a package for constraint-based modeling of biological networks written in Python.

This tool allows loading and inspecting Genome-Scale Metabolic (GEM) models written in the Systems Biology Markup Language (SBML) format.

Using COBRApy, one can analyze the following model contents:
1. Reactions
2. Metabolites
3. Genes
4. Exchange reactions (Environmental Conditions)

COBRApy allows manipulating the contents of a GEM model. For instance, one can edit reactions' flux bounds, knock out a metabolic gene, or change environmental conditions.

COBRApy contains several flux analysis methods to simulate an organism's phenotypic behavior. These include Flux Balance Analysis (FBA), Parsimonious FBA, or Flux Variability Analysis (FVA).

The simulation of gene and reaction deletions for a given GEM model is a straightforward process. One can simulate single or double knockouts using one of the flux analysis methods mentioned above.

## Instalation


### Requirements
The following requirements are needed to use COBRApy:
- Python 3.6 or higher
- pip must be installed
- GLPK is the default solver, but CPLEX is preferred


### How to install COBRApy?
```
pip install cobra
```

# Exercise 3 - Flux Analysis & Constraint-based modeling

## Working with a GEM model

COBRApy can be used to read a GEM model in an SBML format file.
For this practical session, we will be using the following model:
- _E. coli_ core model which contains the central carbon metabolism of _Escherichia coli_ -> file: ../data/e_coli_core.xml

You can read more about _E. coli_ core model (Orth et al 2010) in the following links:
- https://journals.asm.org/doi/10.1128/ecosalplus.10.2.1
- http://bigg.ucsd.edu/models/e_coli_core

The reactions, metabolites, and genes encoded into an SBML format file can be parsed by COBRApy. These contents are loaded into Python objects simple to use, namely `cobra.Reaction`, `cobra.Metabolite`, and `cobra.Gene`

The model itself will be available as an `cobra.Model` object containing all these attributes.

In [ ]:
# importing cobra
import cobra

In [ ]:
# Loading a model
model_path = '../data/e_coli_core.xml'
model = cobra.io.read_sbml_model(model_path)

model

In [ ]:
# retriving the first five reactions of the model
model.reactions[0:5]

In [ ]:
# inspecting the first reaction of the model
model.reactions[0]

In [ ]:
# inspecting a reaction by its ID
model.reactions.get_by_id("PFK")

In [ ]:
# inspecting a reaction using the ID as attribute
model.reactions.PFK

In [ ]:
# retriving the first five metabolites of the model
model.metabolites[0:5]

In [ ]:
# inspecting the first metabolite of the model
model.metabolites[0]

In [ ]:
# inspecting a metabolite by its ID
model.metabolites.get_by_id("glc__D_e")

In [ ]:
# inspecting a metabolite using the ID as attribute
model.metabolites.glc__D_e

In [ ]:
# retriving the first five genes of the model
model.genes[0:5]

In [ ]:
#inspecting the first gene of the model
model.genes[0]

In [ ]:
# inspecting a gene by its ID
model.genes.get_by_id('b1241')

In [ ]:
# inspecting a gene using the ID as attribute
model.genes.b1241

### Reactions

In COBRApy, reactions are objects that can be inspected. A `Reaction` object usually contains the following information:
- name;
- metabolites;
- stoichiometry;
- genes;
- formula;
- reversibility;
- bounds;
- gpr;

In [ ]:
#inspecting the reaction name, formula, metabolites, and stoichiometry.
reaction = model.reactions.get_by_id('ACALD')

print('Reaction name:', reaction.name, '\n')
print('Reaction formula', reaction.reaction, '\n')
for metabolite, coefficient in reaction.metabolites.items():
    print(metabolite, '->', coefficient)

In [ ]:
# inspecting reversibility and flux bounds.
print(reaction.lower_bound, "< ACALD <", reaction.upper_bound, '\n')
print('Reaction reversibility:', reaction.reversibility, '\n')
print('Reaction bounds:', reaction.bounds)

In [ ]:
# change reaction bounds
reaction.bounds = (0, 1000)
print(reaction.lower_bound, "< ACALD <", reaction.upper_bound, '\n')
print('Reaction reversibility:', reaction.reversibility, '\n')

### Metabolites

In COBRApy, metabolites are objects that can be inspected. A `Metabolite` object usually contains the following information:
- name;
- chemical formula;
- compartment;
- reactions;

In [ ]:
# inspecting metabolite name, chemical formula, compartment and reactions.
metabolite = model.metabolites.get_by_id('glc__D_e')

print('Metabolite name:', metabolite.name, '\n')
print('Metabolite chemical formula:', metabolite.formula, '\n')
print('Metabolite compartment:', metabolite.compartment, '\n')
for _reaction in metabolite.reactions:
    print(_reaction.id, ':', _reaction.reaction)

### Genes

In COBRApy, genes are objects that can be inspected. A `Gene` object usually contains the following information:
- name;
- reactions;

In [ ]:
# inspect gene name and reactions.
gene = model.genes.get_by_id('b0351')

print('Gene name:', gene.name, '\n')
for reaction_ in gene.reactions:
    print(reaction_.id, ':', reaction.gene_reaction_rule)

### Exchanges

In COBRApy, exchanges are reaction objects that can be inspected in the model. These special reactions define the environmental condictions (eg. medium) of the model. According to the lower and upper bound of each exchange reaction, one can visualize the model's uptake and secretion metabolites.

In [ ]:
# inspecting the exchange reactions.
for exchange in model.exchanges:
    print(exchange.name, '->', exchange.bounds)

### Phenotype Prediction

COBRApy includes different algorithms for phenotype prediction, such as Flux Balance Analysis (FBA), Parsimonious Flux Balance Analysis (pFBA), and Flux Variability Analysis (FVA).

To perform a simulation using one of these methods, you should first define an objective function. This can be a reaction or an exchange, which will be maximized or minimized. By default, the biomass reaction is set as the model objective function as this mimics the biological behavior of most organisms.

In [ ]:
model.objective.expression

#### Flux Balance Analysis (FBA)

FBA simulations can be performed using `model.optimize()`. This method returns a `Solution` object that contains the following contents:
- objective_value;
- status;
- fluxes;
- reduced costs;

In [ ]:
# performing a FBA simulation
fba_solution = model.optimize()
fba_solution

An FBA simulation can also be performed using the `model.summary()` method.


This method returns the flux value for the reaction defined as the objective function. 

It also returns the flux distribution of the exchange reactions. 
One can see the uptake fluxes corresponding to the rates of metabolite consumption as well as the secretion fluxes corresponding to the rates of metabolite production.

In [ ]:
model.summary()

### Pathway Visualization

COBRApy does not include any pathway visualization method. Nevertheless, independent tools that work with cobra models can be used for that goal. One such tool is named Escher.

Escher allows the visualization of metabolic pathway maps. Escher maps can be visualized using the `escher.Build()` function, which requires the model, the metabolic pathway map, and an FBA solution.

To obtain the metabolic map for the model in question, Escher can be run as follows:

In [ ]:
# first import escher
import escher

In [ ]:
# create the builder object wich contains the metabolic map
builder = escher.Builder(map_name='e_coli_core.Core metabolism', model=model, reaction_data=fba_solution.fluxes)
builder